In [1]:
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import imageio

In [ ]:
# Initialize video capture and background subtractor
capture = cv2.VideoCapture('video4.mp4')
back_sub = cv2.createBackgroundSubtractorMOG2()

# Minimum contour area threshold
min_contour_area = 800

# List to store processed frames
frames = []

# Process the video
while capture.isOpened():
    ret, frame = capture.read()
    if not ret:  # Exit the loop when the video ends
        break

    # Apply background subtraction
    fg_mask = back_sub.apply(frame)

    # Threshold the foreground mask
    retval, mask_thresh = cv2.threshold(fg_mask, 180, 255, cv2.THRESH_BINARY)
    
    # Set the kernel and apply morphological erosion
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_eroded = cv2.morphologyEx(mask_thresh, cv2.MORPH_OPEN, kernel)

    # Find contours in the processed mask
    contours, hierarchy = cv2.findContours(mask_eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours by area
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Draw bounding rectangles
    frame_out = frame.copy()
    for cnt in large_contours:
        x, y, w, h = cv2.boundingRect(cnt)
        frame_out = cv2.rectangle(frame_out, (x, y), (x + w, y + h), (0, 0, 200), 3)  # Bounding box in red

    # Resize the frame for display and save it to the list
    resized_frame = cv2.resize(frame_out, (640, 360))  # Resize to 640x360
    frames.append(cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB))  # Convert to RGB for PIL compatibility

    # # Display the frame
    # cv2.imshow('Frame_final', resized_frame)

    # if cv2.waitKey(1) & 0xFF == ord('q'):  # Break the loop on 'q' key press
    #     break

# Release resources
capture.release()
cv2.destroyAllWindows()


